# GPT2 - Pytorch
This notebook shows how to fine-tune a "gpt2" PyTorch model with AWS Trainium (trn1 instances) using NeuronSDK. The original implementation is provided by HuggingFace.

The example has 2 stages:
1. First compile the model using the utility `neuron_parallel_compile` to compile the model to run on the AWS Trainium device.
1. Run the fine-tuning script to train the model based on causal language modeling (CLM) loss. The training job will use 2 workers with data parallel to speed up the training. If you have a larger instance (trn1.32xlarge) you can increase the worker count to 8 or 32.

It has been tested and run on a trn1.2xlarge

**Reference:** https://huggingface.co/gpt2

## 1) Install dependencies

In [ ]:
# Set Pip repository  to point to the Neuron repository
%pip config set global.extra-index-url https://pip.repos.neuron.amazonaws.com
# now restart the kernel

In [ ]:
#Install Neuron Compiler and Neuron/XLA packages
%pip install -U torch-neuronx=="1.11.0.1.*" "numpy<=1.20.0" "protobuf<4" "transformers==4.16.2" datasets sklearn
# use --force-reinstall if you're facing some issues while loading the modules
# now restart the kernel again

## 2) Set the parameters

In [1]:
# Parameters
model_name = "gpt2"
extra_pip_packages = ""
extra_yum_packages = ""
env_var_options = "XLA_USE_BF16=1 NEURON_CC_FLAGS=\"--cache_dir=./compiler_cache --model-type=transformer\""
num_workers = 2
task_name = "clm"
dataset_name = "wikitext"
dataset_config_name = "wikitext-2-raw-v1"
work_dir = "/home/ec2-user/language_modeling"
transformers_version = "4.16.2"
model_base_name = "gpt2"

## 3) Compile the model with neuron_parallel_compile

In [2]:

print("Compile model")
COMPILE_CMD = f"""{env_var_options} neuron_parallel_compile torchrun --nproc_per_node={num_workers} ./run_clm.py \
    --model_name_or_path {model_name} \
    --dataset_name {dataset_name} \
    --dataset_config_name {dataset_config_name} \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 4 \
    --do_train \
    --overwrite_output_dir \
    --output_dir {model_base_name}-{task_name} |& tee log_compile_{model_base_name}-{task_name}-precompile"""

print(f'Running command: \n{COMPILE_CMD}')
! {COMPILE_CMD}


Compile model
Running command: 
XLA_USE_BF16=1 NEURON_CC_FLAGS="--cache_dir=./compiler_cache --model-type=transformer" neuron_parallel_compile torchrun --nproc_per_node=2 ./run_clm.py     --model_name_or_path gpt2     --dataset_name wikitext     --dataset_config_name wikitext-2-raw-v1     --per_device_train_batch_size 4     --per_device_eval_batch_size 4     --do_train     --overwrite_output_dir     --output_dir gpt2-clm |& tee log_compile_gpt2-clm-precompile
2022-10-20 08:52:16.000555: INFO ||PARALLEL_COMPILE||: Running trial run (add option to terminate trial run early; also ignore trial run's generated outputs, i.e. loss, checkpoints)
*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
__main__: Process rank: 1, device: xla:0, n_gpu: 0distribute

## 4) Fine-tune the model

In [3]:
print("Train model")
RUN_CMD = f"""{env_var_options} torchrun --nproc_per_node={num_workers} run_clm.py \
    --model_name_or_path {model_name} \
    --dataset_name {dataset_name} \
    --dataset_config_name {dataset_config_name} \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 4 \
    --do_train \
    --do_eval \
    --overwrite_output_dir \
    --output_dir {model_base_name}-{task_name} |& tee log_run_{model_base_name}-{task_name}"""

print(f'Running command: \n{RUN_CMD}')
! {RUN_CMD}

Train model
Running command: 
XLA_USE_BF16=1 NEURON_CC_FLAGS="--cache_dir=./compiler_cache --model-type=transformer" torchrun --nproc_per_node=2 run_clm.py     --model_name_or_path gpt2     --dataset_name wikitext     --dataset_config_name wikitext-2-raw-v1     --per_device_train_batch_size 4     --per_device_eval_batch_size 4     --do_train     --do_eval     --overwrite_output_dir     --output_dir gpt2-clm |& tee log_run_gpt2-clm
*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
__main__: Process rank: 1, device: xla:0, n_gpu: 0distributed training: True, 16-bits training: False
__main__: Process rank: 0, device: xla:1, n_gpu: 0distributed training: True, 16-bits training: False
__main__: Training/evaluation parameters TrainingArguments(
_n_gpu=